![Binary code with a magnifying glass](binary.jpg)

As a data engineer, you often face unexpected challenges in workflows. In this scenario, the `load_and_check()` function, in charge of managing sales data, encounters issues after the latest update. Unfortunately, your colleague who usually handles this code is currently on holiday, leaving you to troubleshoot.

Your task is to identify and address the issues in the sales data pipeline **without getting into every line of code.** The `load_and_check()` function loads the `sales.csv` dataset and performs several checks. Initially, it verifies the dataset's shape, ensuring it matches expectations. Subsequently, integrity checks are conducted to maintain data consistency and flag any anomalies.

The `sales.csv` dataset has various columns, focusing on critical fields such as `Total`, `Quantity`, `Unit price`, `Tax`, and `Date`. It's essential that the `Tax` column accurately represents 5% of the subtotal, calculated from the `Unit Price` multiplied by `Quantity`.

**Your goal is to sort out the pipeline issues, aiming for the code to return 2 success messages upon completion.** While at it, try to keep the original structure as much as possible. Only change existing columns if necessary, and make sure the data remains accurate. Be mindful of updating any relevant if statements in the checks as needed.

In [32]:
import pandas as pd

def load_and_check():
    # Step 1: Load the data and check if it has the expected shape
    data = pd.read_csv('sales.csv')  
    
    if data.shape[1] != 17:
        print("Please check that the data was loaded properly!")
    else:
        print("Data loaded successfully.")

    # Step 2: Calculate statistical values and merge with the original data
    grouped_data = data.groupby(['Date'])['Total'].agg(['mean', 'std'])
    grouped_data['threshold'] = 3 * grouped_data['std']
    grouped_data['max'] = grouped_data['mean'] + grouped_data.threshold
    grouped_data['min'] = grouped_data[['mean', 'threshold']].apply(lambda row: max(0, row['mean'] - row['threshold']), axis=1)
    data = pd.merge(data, grouped_data, on='Date', how='left')
    data['Tax'] = data['Quantity'] * data['Unit price'] * 0.05

    # Condition_1 checks if 'Total' is within the acceptable range (min to max) for each date
    data['Condition_1'] = (data['Total'] >= data['min']) & (data['Total'] <= data['max'])
    data['Condition_1'].fillna(False, inplace=True)  

    # Condition_2 checks if the 'Tax' column is properly calculated as 5% of (Quantity * Unit price)
    data['Condition_2'] = round(data['Quantity'] * data['Unit price'] * 0.05, 1) == round(data['Tax'], 1)
        
    # Step 3: Check if all rows pass both Condition_1 and Condition_2
    # Success indicates data integrity; failure suggests potential issues.
    if (data['Condition_1'].sum() == data.shape[0]) and (data['Condition_2'].sum() == data.shape[0]):
        print("Data integrity check was successful! All rows pass the integrity conditions.")
    else:
        print("Something fishy is going on with the data! Integrity check failed for some rows!")
        
    return data

processed_data = load_and_check()

Data loaded successfully.
Data integrity check was successful! All rows pass the integrity conditions.


In [33]:
load_and_check().head()

Data loaded successfully.
Data integrity check was successful! All rows pass the integrity conditions.


,Invoice ID,Branch,City,Customer type,Gender,Product line,Unit price,Quantity,Tax,Total,Date,Time,Payment,cogs,gross margin percentage,gross income,Rating,mean,std,threshold,max,min,Condition_1,Condition_2
0,101-17-6199,A,Yangon,Normal,Male,Food and beverages,45.79,7,16.0265,336.5565,3/13/2019,19:44:00.0,Credit card,320.53,4.761905,16.0265,7.0,206.360700,145.045037,435.135112,641.495812,0,True,True
1,101-81-4070,C,Naypyitaw,Member,Female,Health and beauty,62.82,2,6.2820,131.9220,1/17/2019,12:36:00.0,Ewallet,125.64,4.761905,6.2820,4.9,285.705000,270.978942,812.936827,1098.641827,0,True,True
2,102-06-2002,C,Naypyitaw,Member,Male,Sports and travel,25.25,5,6.3125,132.5625,3/20/2019,17:52:00.0,Cash,126.25,4.761905,6.3125,6.1,363.880300,265.863137,797.589410,1161.469710,0,True,True
3,102-77-2261,C,Naypyitaw,Member,Male,Health and beauty,65.31,7,22.8585,480.0285,3/5/2019,18:02:00.0,Credit card,457.17,4.761905,22.8585,4.2,366.522265,250.589063,751.767190,1118.289455,0,True,True
4,105-10-6182,A,Yangon,Member,Male,Fashion accessories,21.48,2,2.1480,45.1080,2/27/2019,12:22:00.0,Ewallet,42.96,4.761905,2.1480,6.6,418.532250,274.813848,824.441545,1242.973795,0,True,True
